In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('./features/feature_zero.csv')

train.shape
train.head()

(331873, 1099)

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,ordadvanceday,masterbasicroomid,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,noroom,orderdate_mon,arrival_mon,etd_mon,orderdate_day,arrival_day,etd_day,totalrooms_x,glon,glat,totalrooms_y,actual_day,order_day,is_equal,hasroom,city_hot,countryid_hot,hotel_hot,room_hot,masterbasicroomid_hot,masterhotelid_hot,supplierid_hot,isholdroom_pos_hot,isholdroom_nag_hot,ordadvanceday_pos_hot,ordadvanceday_nag_hot,isvendor_pos_hot,isvendor_nag_hot,hotelbelongto_pos_hot,hotelbelongto_nag_hot,isebookinghtl_pos_hot,isebookinghtl_nag_hot,hotelstar_pos_hot,hotelstar_nag_hot,supplierchannel_pos_hot,supplierchannel_nag_hot,orderdate_mon_pos_hot,orderdate_mon_nag_hot,arrival_mon_pos_hot,arrival_mon_nag_hot,etd_mon_pos_hot,etd_mon_nag_hot,pos_city_isholdroom_mean,nag_city_isholdroom_mean,pos_city_isholdroom_min,nag_city_isholdroom_min,pos_city_isholdroom_max,nag_city_isholdroom_max,pos_city_isholdroom_var,nag_city_isholdroom_var,pos_city_isholdroom_median,nag_city_isholdroom_median,pos_city_isholdroom_mean_sub,pos_city_isholdroom_median_sub,nag_city_isholdroom_mean_sub,nag_city_isholdroom_median_sub,pos_countryid_isholdroom_mean,nag_countryid_isholdroom_mean,pos_countryid_isholdroom_min,nag_countryid_isholdroom_min,pos_countryid_isholdroom_max,nag_countryid_isholdroom_max,pos_countryid_isholdroom_var,nag_countryid_isholdroom_var,pos_countryid_isholdroom_median,nag_countryid_isholdroom_median,pos_countryid_isholdroom_mean_sub,pos_countryid_isholdroom_median_sub,nag_countryid_isholdroom_mean_sub,nag_countryid_isholdroom_median_sub,pos_hotel_isholdroom_mean,nag_hotel_isholdroom_mean,pos_hotel_isholdroom_min,nag_hotel_isholdroom_min,pos_hotel_isholdroom_max,nag_hotel_isholdroom_max,pos_hotel_isholdroom_var,nag_hotel_isholdroom_var,pos_hotel_isholdroom_median,nag_hotel_isholdroom_median,pos_hotel_isholdroom_mean_sub,pos_hotel_isholdroom_median_sub,nag_hotel_isholdroom_mean_sub,nag_hotel_isholdroom_median_sub,pos_room_isholdroom_mean,nag_room_isholdroom_mean,pos_room_isholdroom_min,nag_room_isholdroom_min,pos_room_isholdroom_max,nag_room_isholdroom_max,pos_room_isholdroom_var,nag_room_isholdroom_var,pos_room_isholdroom_median,nag_room_isholdroom_median,pos_room_isholdroom_mean_sub,pos_room_isholdroom_median_sub,nag_room_isholdroom_mean_sub,nag_room_isholdroom_median_sub,pos_masterbasicroomid_isholdroom_mean,nag_masterbasicroomid_isholdroom_mean,pos_masterbasicroomid_isholdroom_min,nag_masterbasicroomid_isholdroom_min,pos_masterbasicroomid_isholdroom_max,nag_masterbasicroomid_isholdroom_max,pos_masterbasicroomid_isholdroom_var,nag_masterbasicroomid_isholdroom_var,pos_masterbasicroomid_isholdroom_median,nag_masterbasicroomid_isholdroom_median,pos_masterbasicroomid_isholdroom_mean_sub,pos_masterbasicroomid_isholdroom_median_sub,nag_masterbasicroomid_isholdroom_mean_sub,nag_masterbasicroomid_isholdroom_median_sub,pos_masterhotelid_isholdroom_mean,nag_masterhotelid_isholdroom_mean,pos_masterhotelid_isholdroom_min,nag_masterhotelid_isholdroom_min,pos_masterhotelid_isholdroom_max,nag_masterhotelid_isholdroom_max,pos_masterhotelid_isholdroom_var,nag_masterhotelid_isholdroom_var,pos_masterhotelid_isholdroom_median,nag_masterhotelid_isholdroom_median,pos_masterhotelid_isholdroom_mean_sub,pos_masterhotelid_isholdroom_median_sub,nag_masterhotelid_isholdroom_mean_sub,nag_masterhotelid_isholdroom_median_sub,pos_supplierid_isholdroom_mean,nag_supplierid_isholdroom_mean,pos_supplierid_isholdroom_min,nag_supplierid_isholdroom_min,pos_supplierid_isholdroom_max,nag_supplierid_isholdroom_max,pos_supplierid_isholdroom_var,nag_supplierid_isholdroom_var,pos_supplierid_isholdroom_median,nag_supplierid_isholdroom_median,pos_supplierid_isholdroom_mean_sub,pos_supplierid_isholdroom_median_sub,nag_supplierid_isholdroom_mean_sub,nag_supplierid_isholdroom_median_sub,pos_city_ordadvanceday_mean,nag_city_ordadvanceday_mean,pos_city_ordadvanceday_min,nag_city_ordadvanceday

In [3]:
import xgboost as xgb
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score

In [4]:
basic_fea = ['orderid','city','countryid','hotel','zone','room','isholdroom','ordadvanceday','masterbasicroomid','masterhotelid','supplierid','isvendor','isebookinghtl','hotelstar','supplierchannel','orderdate_mon','arrival_mon','etd_mon','orderdate_day','arrival_day','etd_day','totalrooms_x','glon','glat','totalrooms_y','actual_day','order_day','is_equal']
basic_fea_noid = ['isholdroom','ordadvanceday','isvendor','isebookinghtl','hotelstar','supplierchannel','orderdate_mon','arrival_mon','etd_mon','orderdate_day','arrival_day','etd_day','totalrooms_x','glon','glat','totalrooms_y','actual_day','order_day','is_equal']
target = ['noroom']
all_fea = list(set(train.columns) - set(basic_fea + ['noroom', 'hasroom']))

cand_fea = []
for i in train.columns:
    if '_' in i:
        cand_fea.append(i)
cand_fea = list(set(cand_fea) - set(basic_fea))
len(cand_fea)

cand_fea2 = []
for i in train.columns:
    if 'date_sub' in i or '_num' in i or 'daysum' in i:
        cand_fea2.append(i)
len(cand_fea2)

1065

44

In [5]:
train[basic_fea] = train[basic_fea].astype(float)

test = train[train.noroom == -1]
train = train[train.noroom != -1]

train.shape
test.shape

(309802, 1099)

(22071, 1099)

## Step1

In [7]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    precision, recall, thresholds = precision_recall_curve(labels, preds)
    recall, precision = recall[::-1], precision[::-1]
    area = 0
    had_find_r_start = False
    for idx, item in enumerate(zip(recall, precision)):
        r, p = item[0], item[1]
        if r >= 0.5:
            area += precision[idx-1] * (0.5 - recall[idx-1])
            return 'prc', -area
        if r > 0.05:
            if idx == 0:
                area += precision[idx] * (r - 0.05)
            else:           
                area += precision[idx-1] * (r - recall[idx-1])
    return 'prc', -area

In [8]:
%%time
params = {
    'objective': 'binary:logistic',
    'eta': 0.1,
    'colsample_bytree': 0.886,
    'min_child_weight': 0.1,
    'max_depth': 15,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'seed': 2018,
    'missing': -1
}

len(basic_fea)
xgbtrain = xgb.DMatrix(train[basic_fea], train[target])

print('Start training')
cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=100,
    nfold=3,
    feval=evalerror,
    verbose_eval=100,
    early_stopping_rounds=100,
    seed=2018)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-prc-mean']))
print('End training')

Start training
[0]	train-error:0.039737+0.0013607	train-prc:-0.180913+0.0400324	test-error:0.0435793+0.000405628	test-prc:-0.147625+0.0366747
[99]	train-error:0.0260747+0.000341423	train-prc:-0.440574+0.000709956	test-error:0.0398543+0.000579811	test-prc:-0.278165+0.00301131
100:-0.2781653333333333
End training
CPU times: user 3h 25min 35s, sys: 11.9 s, total: 3h 25min 47s
Wall time: 10min 9s


In [ ]:
basic_fea:100:0.2750456666666667

In [11]:
finish_fea = []
for k, v in feature_preformance.items():
    finish_fea.append(k)
len(finish_fea)

0

In [10]:
feature_preformance = {}
params = {
    'objective': 'binary:logistic',
    'eta': 0.1,
    'colsample_bytree': 0.886,
    'min_child_weight': 0.1,
    'max_depth': 15,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'seed': 2018,
    'missing': -1
}

for i in tqdm_notebook(['zq_num', 'bk_num']):
    if i not in finish_fea:
        tmp_predictors = basic_fea.copy()
        tmp_predictors.append(i)
        assert(len(tmp_predictors) == 28)
        xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])
        cv = xgb.cv(
        params,
        dtrain=xgbtrain,
        num_boost_round=100,
        nfold=3,
        feval=evalerror,
        verbose_eval=0,
        early_stopping_rounds=100,
        seed=2018)
        feature_preformance[i] = -cv.iloc[-1, :]['test-prc-mean']
        finish_fea.append(i)

In [36]:
feature_preformance

{'bk_num': 0.27546133333333334, 'zq_num': 0.2760476666666667}

In [27]:
good_feature = []
baseline = 0.2750456666666667

for k,v in sorted(feature_preformance.items(), key=lambda x:x[1], reverse=True):
    if v > baseline:
        good_feature.append(k)
len(good_feature)

26

## Step2

In [7]:
feature_performance={'nag_room_ordadvanceday_mean':0.274623,'pos_supplierid_isvendor_min':0.27604199999999995,'nag_city_hotelbelongto_median_sub':0.2758066666666667,'nag_hotel_orderdate_mon_var':0.2777366666666667,'pos_masterhotelid_supplierchannel_var':0.27714733333333336,'nag_city_isvendor_min':0.27577633333333335,'pos_masterbasicroomid_hotelbelongto_mean':0.27560233333333334,'nag_hotel_hotelbelongto_mean':0.276386,'nag_city_orderdate_mon_var':0.27469866666666665,'pos_hotel_etd_mon_mean_sub':0.27800933333333333,'pos_city_supplierchannel_mean':0.275886,'pos_hotel_supplierchannel_max':0.2773113333333333,'nag_hotel_hotelstar_mean_sub':0.27764633333333333,'pos_masterhotelid_ordadvanceday_mean_sub':0.2740553333333334,'pos_supplierid_ordadvanceday_min':0.27540733333333334,'pos_masterbasicroomid_supplierchannel_max':0.275669,'pos_masterbasicroomid_orderdate_mon_var':0.27535200000000004,'nag_supplierid_arrival_mon_median_sub':0.275399,'nag_supplierid_hotelbelongto_median_sub':0.27543066666666666,'pos_masterhotelid_hotelstar_mean_sub':0.27549966666666664,'nag_countryid_isebookinghtl_mean_sub':0.2762153333333333,'pos_room_isebookinghtl_min':0.27604666666666666,'pos_supplierid_orderdate_mon_mean_sub':0.276778,'nag_countryid_isholdroom_var':0.2762853333333333,'nag_city_supplierchannel_mean':0.2751596666666667,'pos_masterhotelid_isebookinghtl_mean_sub':0.27669366666666667,'pos_room_isholdroom_min':0.27563699999999997,'pos_countryid_isebookinghtl_median':0.27615266666666666,'pos_hotel_isebookinghtl_var':0.27649266666666666,'isebookinghtl_pos_hot':0.27528566666666665,'pos_city_isholdroom_max':0.27574733333333334,'nag_countryid_isvendor_median':0.27529,'pos_hotel_isvendor_var':0.27692433333333333,'nag_countryid_isvendor_var':0.27467933333333333,'nag_city_etd_mon_median':0.27559666666666666,'pos_hotel_hotelbelongto_min':0.27786,'pos_hotel_hotelstar_median':0.27704466666666666,'nag_hotel_supplierchannel_min':0.277642,'pos_supplierid_etd_mon_mean_sub':0.275966,'nag_supplierid_isholdroom_mean_sub':0.27651933333333334,'pos_hotel_supplierchannel_mean':0.2773113333333333,'nag_city_isholdroom_max':0.2760266666666667,'nag_hotel_orderdate_mon_mean_sub':0.2776133333333333,'nag_city_arrival_mon_median':0.27518566666666666,'pos_supplierid_hotelbelongto_min':0.27652366666666667,'pos_countryid_isebookinghtl_mean_sub':0.2759416666666667,'pos_masterhotelid_isebookinghtl_median_sub':0.27634533333333333,'nag_room_isvendor_min':0.276051,'pos_countryid_supplierchannel_median_sub':0.27565466666666666,'pos_supplierid_orderdate_mon_max':0.2760223333333333,'pos_masterbasicroomid_arrival_mon_var':0.2751996666666667,'pos_room_hotelstar_var':0.27457233333333336,'pos_hotel_etd_mon_median_sub':0.2773446666666667,'arrival_mon_nag_hot':0.27562533333333333,'nag_masterbasicroomid_etd_mon_min':0.27485000000000004,'nag_masterhotelid_ordadvanceday_median':0.275168,'nag_hotel_ordadvanceday_min':0.27591933333333335,'nag_hotel_isebookinghtl_min':0.2766946666666666,'nag_masterhotelid_supplierchannel_min':0.27645533333333333,'nag_countryid_hotelbelongto_min':0.27576033333333333,'nag_masterbasicroomid_supplierchannel_max':0.275431,'nag_countryid_ordadvanceday_median':0.27566633333333335,'nag_room_isholdroom_median':0.2748793333333333,'pos_supplierid_isebookinghtl_max':0.27511766666666665,'nag_hotel_hotelstar_max':0.275517,'pos_masterhotelid_supplierchannel_median_sub':0.27676466666666666,'nag_masterbasicroomid_etd_mon_var':0.275851,'pos_hotel_isvendor_mean_sub':0.27854033333333333,'nag_supplierid_isvendor_median':0.2765303333333333,'pos_supplierid_isholdroom_median':0.276003,'nag_masterbasicroomid_hotelbelongto_median_sub':0.276389,'pos_masterbasicroomid_hotelstar_max':0.275276,'nag_city_arrival_mon_min':0.27522566666666665,'pos_countryid_etd_mon_mean':0.2751253333333333,'pos_masterbasicroomid_hotelstar_median':0.2759593333333334,'pos_masterbasicroomid_hotelbelongto_min':0.2755183333333333,'pos_masterhotelid_hotelstar_mean':0.27568066666666663,'pos_city_hotelstar_median_sub':0.27575666666666665,'pos_room_supplierchannel_var':0.27457233333333336,'nag_masterhotelid_isvendor_median':0.2755163333333333,'nag_room_orderdate_mon_median_sub':0.275105,'pos_masterhotelid_orderdate_mon_var':0.27606533333333333,'pos_masterhotelid_supplierchannel_mean_sub':0.276254,'pos_masterhotelid_isvendor_max':0.276715,'pos_room_ordadvanceday_var':0.27162233333333335,'pos_city_hotelbelongto_max':0.2752206666666667,'pos_countryid_arrival_mon_var':0.27524433333333337,'pos_masterbasicroomid_hotelbelongto_median':0.2756706666666667,'pos_masterhotelid_isholdroom_mean_sub':0.27515266666666666,'pos_city_etd_mon_median':0.2746753333333333,'pos_countryid_ordadvanceday_min':0.27536999999999995,'pos_supplierid_isebookinghtl_median':0.2753893333333333,'pos_countryid_supplierchannel_min':0.27612966666666666,'pos_hotel_orderdate_mon_mean_sub':0.278054,'pos_masterhotelid_ordadvanceday_min':0.27546533333333334,'pos_countryid_etd_mon_median':0.27528933333333333,'masterhotelid_hot':0.27667166666666665,'pos_supplierid_supplierchannel_median':0.2745666666666667,'nag_city_hotelstar_median_sub':0.27527833333333335,'nag_room_isholdroom_max':0.275773,'nag_city_supplierchannel_median':0.27557366666666666,'pos_countryid_hotelbelongto_var':0.2760073333333333,'nag_supplierid_isvendor_mean_sub':0.27529166666666666,'nag_city_arrival_mon_mean_sub':0.2754233333333333,'pos_hotel_supplierchannel_var':0.27649266666666666,'pos_room_hotelstar_max':0.276022,'pos_city_etd_mon_median_sub':0.27557533333333334,'pos_room_isholdroom_median':0.2764236666666667,'pos_city_isvendor_mean':0.275536,'pos_city_hotelstar_var':0.27563266666666664,'nag_supplierid_hotelbelongto_max':0.27536466666666665,'nag_masterhotelid_etd_mon_max':0.2783,'pos_room_orderdate_mon_median':0.2759396666666667,'pos_hotel_isvendor_median_sub':0.27723499999999995,'nag_countryid_isvendor_max':0.27529166666666666,'pos_supplierid_arrival_mon_min':0.2756173333333333,'nag_masterhotelid_isholdroom_var':0.276082,'pos_city_isholdroom_min':0.27538233333333334,'nag_masterhotelid_etd_mon_median_sub':0.27594566666666664,'pos_room_etd_mon_max':0.276331,'nag_city_isvendor_median_sub':0.27617,'nag_masterhotelid_supplierchannel_max':0.276129,'nag_room_orderdate_mon_mean':0.27518333333333334,'pos_room_ordadvanceday_max':0.27537133333333336,'nag_countryid_isholdroom_max':0.275031,'hotelstar_nag_hot':0.27605433333333335,'pos_countryid_ordadvanceday_mean_sub':0.27511033333333335,'nag_room_orderdate_mon_min':0.27564,'nag_countryid_isholdroom_median':0.27538233333333334,'nag_supplierid_isebookinghtl_mean':0.2763623333333333,'pos_room_supplierchannel_mean':0.27659100000000003,'nag_masterhotelid_supplierchannel_mean_sub':0.2771436666666667,'etd_mon_pos_hot':0.27489499999999994,'pos_supplierid_hotelstar_median_sub':0.2753936666666666,'pos_masterhotelid_ordadvanceday_var':0.27582966666666664,'nag_room_supplierchannel_var':0.2749486666666667,'nag_city_ordadvanceday_mean':0.2758903333333333,'pos_masterhotelid_isvendor_var':0.27715,'pos_supplierid_hotelbelongto_mean_sub':0.2755323333333333,'pos_supplierid_hotelstar_mean':0.2764396666666667,'pos_city_arrival_mon_mean_sub':0.27613466666666664,'nag_masterbasicroomid_arrival_mon_max':0.2747903333333333,'pos_masterbasicroomid_etd_mon_median_sub':0.27520266666666665,'pos_room_ordadvanceday_median_sub':0.2730283333333334,'nag_supplierid_ordadvanceday_median_sub':0.27529766666666666,'nag_masterhotelid_hotelstar_var':0.275073,'nag_masterbasicroomid_ordadvanceday_median':0.2753623333333333,'nag_city_hotelbelongto_mean_sub':0.275665,'ordadvanceday_pos_hot':0.275106,'supplierid_hot':0.27753066666666665,'pos_masterbasicroomid_orderdate_mon_median_sub':0.275688,'nag_countryid_hotelbelongto_mean':0.2755396666666667,'pos_hotel_supplierchannel_median_sub':0.2784356666666667,'nag_masterhotelid_isholdroom_mean':0.276306,'pos_masterbasicroomid_isebookinghtl_mean_sub':0.27620500000000003,'nag_room_ordadvanceday_max':0.27527533333333337,'pos_room_hotelstar_mean_sub':0.27637066666666665,'pos_room_isvendor_mean':0.27513666666666664,'pos_countryid_hotelstar_mean_sub':0.2764833333333333,'nag_supplierid_isholdroom_max':0.2757716666666667,'nag_hotel_hotelstar_median_sub':0.27764633333333333,'pos_room_isholdroom_max':0.27718,'nag_masterbasicroomid_hotelstar_min':0.27581533333333336,'nag_supplierid_orderdate_mon_max':0.2750696666666667,'nag_city_arrival_mon_median_sub':0.27651033333333336,'pos_supplierid_isvendor_median':0.27571433333333334,'pos_supplierid_ordadvanceday_median':0.2764856666666667,'nag_room_isebookinghtl_median':0.275455,'nag_countryid_orderdate_mon_var':0.2756893333333333,'nag_hotel_ordadvanceday_median':0.27667766666666666,'nag_city_isholdroom_median':0.27543666666666666,'nag_masterbasicroomid_isholdroom_mean_sub':0.2751303333333333,'pos_supplierid_isebookinghtl_var':0.27477100000000004,'pos_countryid_hotelstar_min':0.27534733333333333,'pos_room_orderdate_mon_max':0.2764486666666666,'nag_masterbasicroomid_orderdate_mon_median':0.27539399999999997,'pos_room_arrival_mon_median':0.2760723333333333,'nag_masterhotelid_orderdate_mon_max':0.2760106666666667,'nag_city_hotelstar_mean_sub':0.2755013333333334,'nag_countryid_arrival_mon_mean':0.275777,'pos_room_supplierchannel_mean_sub':0.27637066666666665,'nag_masterhotelid_orderdate_mon_mean':0.2769506666666666,'pos_masterhotelid_isvendor_median':0.2764706666666667,'pos_countryid_isebookinghtl_min':0.2751,'nag_city_etd_mon_max':0.2749996666666667,'pos_supplierid_isholdroom_min':0.2757176666666667,'nag_countryid_arrival_mon_var':0.274325,'nag_hotel_hotelbelongto_min':0.276386,'nag_city_ordadvanceday_min':0.27523133333333333,'nag_room_isvendor_median_sub':0.276321,'nag_city_etd_mon_mean':0.27594599999999997,'nag_room_isvendor_var':0.2749486666666667,'pos_city_hotelstar_median':0.27545233333333335,'pos_city_ordadvanceday_mean':0.275883,'nag_masterhotelid_hotelstar_median_sub':0.27624366666666667,'pos_hotel_hotelstar_median_sub':0.2784356666666667,'nag_room_hotelbelongto_var':0.2749486666666667,'pos_room_isebookinghtl_var':0.27457233333333336,'nag_hotel_orderdate_mon_mean':0.27825000000000005,'nag_hotel_isebookinghtl_mean_sub':0.27764633333333333,'pos_room_orderdate_mon_min':0.27568600000000004,'pos_city_arrival_mon_median_sub':0.27555966666666665,'nag_supplierid_orderdate_mon_var':0.276717,'nag_hotel_arrival_mon_var':0.2771966666666667,'pos_supplierid_arrival_mon_mean':0.27674333333333334,'pos_room_hotelbelongto_median':0.2761636666666667,'nag_room_hotelstar_min':0.2756113333333333,'pos_countryid_orderdate_mon_var':0.2758036666666667,'orderdate_mon_pos_hot':0.2750456666666667,'nag_city_hotelbelongto_max':0.27474366666666666,'pos_city_supplierchannel_min':0.27532366666666663,'pos_masterhotelid_hotelstar_max':0.27609733333333336,'nag_masterhotelid_hotelbelongto_var':0.2769453333333333,'pos_supplierid_isholdroom_var':0.27620266666666665,'nag_city_orderdate_mon_min':0.274742,'pos_city_ordadvanceday_median':0.27502299999999996,'pos_countryid_orderdate_mon_mean':0.27590533333333334,'nag_room_arrival_mon_min':0.2750266666666667,'pos_hotel_etd_mon_min':0.27817566666666665,'pos_city_isholdroom_median_sub':0.2759023333333333,'pos_supplierid_etd_mon_max':0.27571266666666666,'nag_room_hotelbelongto_mean_sub':0.276321,'nag_countryid_isebookinghtl_mean':0.27556633333333336,'nag_hotel_isebookinghtl_mean':0.2766946666666666,'nag_masterbasicroomid_orderdate_mon_max':0.275528,'nag_masterhotelid_orderdate_mon_var':0.27666166666666664,'supplierchannel_nag_hot':0.2764236666666667,'pos_room_hotelbelongto_mean':0.2761636666666667,'pos_room_isholdroom_mean_sub':0.275365,'nag_masterbasicroomid_etd_mon_mean_sub':0.27459066666666665,'pos_supplierid_hotelbelongto_median_sub':0.27504433333333334,'nag_hotel_isvendor_var':0.2777686666666667,'nag_city_etd_mon_var':0.2754006666666666,'pos_masterhotelid_etd_mon_mean_sub':0.2735176666666667,'nag_city_isholdroom_median_sub':0.275326,'pos_masterbasicroomid_hotelbelongto_mean_sub':0.27608499999999997,'nag_hotel_hotelbelongto_max':0.276386,'pos_room_isebookinghtl_mean':0.27604666666666666,'nag_room_supplierchannel_min':0.2752113333333333,'nag_masterhotelid_hotelstar_mean':0.27560266666666666,'nag_masterbasicroomid_etd_mon_max':0.27482233333333334,'nag_supplierid_arrival_mon_median':0.27635800000000005,'pos_countryid_hotelbelongto_median_sub':0.27619066666666664,'nag_countryid_supplierchannel_median_sub':0.2754493333333334,'pos_room_ordadvanceday_min':0.275879,'nag_hotel_isebookinghtl_median_sub':0.27764633333333333,'nag_room_isebookinghtl_median_sub':0.276321,'nag_masterbasicroomid_hotelstar_median':0.27415433333333333,'nag_city_hotelstar_min':0.27578966666666666,'pos_city_etd_mon_max':0.2751476666666667,'nag_room_etd_mon_max':0.27504466666666666,'pos_masterhotelid_supplierchannel_max':0.27576766666666663,'pos_room_isvendor_median_sub':0.27637066666666665,'arrival_mon_pos_hot':0.2757636666666667,'nag_city_isebookinghtl_mean_sub':0.2758313333333334,'pos_masterbasicroomid_isebookinghtl_max':0.27510033333333334,'pos_hotel_orderdate_mon_min':0.27727066666666667,'pos_hotel_hotelstar_min':0.27704466666666666,'pos_room_hotelstar_median':0.276022,'nag_room_isebookinghtl_max':0.275455,'nag_hotel_isebookinghtl_median':0.2766946666666666,'nag_room_hotelbelongto_min':0.275548,'pos_supplierid_etd_mon_min':0.27615933333333337,'nag_supplierid_hotelbelongto_mean':0.276002,'pos_hotel_isvendor_mean':0.277298,'pos_city_isholdroom_median':0.27512166666666665,'pos_countryid_hotelstar_median':0.27568933333333334,'nag_countryid_hotelstar_median_sub':0.27414133333333335,'pos_countryid_hotelbelongto_mean':0.2750776666666667,'nag_hotel_isholdroom_max':0.277329,'nag_room_ordadvanceday_median':0.274719,'nag_masterbasicroomid_arrival_mon_mean_sub':0.27513499999999996,'pos_masterhotelid_arrival_mon_median':0.2753623333333333,'pos_masterbasicroomid_ordadvanceday_median_sub':0.2760293333333333,'pos_city_supplierchannel_mean_sub':0.27548866666666666,'nag_masterbasicroomid_arrival_mon_mean':0.2754473333333333,'nag_countryid_etd_mon_median':0.2759216666666666,'pos_supplierid_etd_mon_mean':0.27647399999999994,'pos_masterbasicroomid_isvendor_min':0.27540000000000003,'nag_city_isebookinghtl_min':0.27592666666666665,'pos_room_ordadvanceday_mean':0.27492833333333333,'nag_supplierid_ordadvanceday_mean':0.27628,'pos_supplierid_ordadvanceday_max':0.27623566666666666,'pos_hotel_isebookinghtl_min':0.2769176666666667,'nag_hotel_isvendor_mean_sub':0.2758936666666667,'nag_room_supplierchannel_mean':0.2752113333333333,'nag_supplierid_isebookinghtl_mean_sub':0.27554866666666666,'pos_supplierid_isvendor_mean_sub':0.27595,'nag_supplierid_orderdate_mon_median_sub':0.2766016666666667,'pos_masterhotelid_arrival_mon_mean':0.2752196666666667,'nag_countryid_arrival_mon_max':0.27521799999999996,'nag_hotel_arrival_mon_mean':0.277358,'pos_city_etd_mon_mean_sub':0.27483300000000005,'pos_city_hotelbelongto_var':0.2749753333333333,'pos_city_ordadvanceday_median_sub':0.2760876666666667,'nag_room_etd_mon_var':0.2749623333333333,'nag_masterhotelid_hotelbelongto_median_sub':0.2762883333333333,'pos_room_supplierchannel_median_sub':0.27637066666666665,'pos_masterbasicroomid_arrival_mon_median_sub':0.2756,'nag_room_supplierchannel_median':0.2752113333333333,'nag_countryid_isvendor_median_sub':0.275641,'nag_city_hotelstar_mean':0.276059,'pos_supplierid_supplierchannel_min':0.27546766666666667,'nag_masterbasicroomid_isholdroom_max':0.2749283333333334,'pos_countryid_isvendor_var':0.275281,'pos_hotel_arrival_mon_max':0.27804433333333334,'pos_supplierid_hotelstar_min':0.2758753333333333,'pos_masterbasicroomid_isvendor_mean_sub':0.27515033333333333,'nag_masterhotelid_isholdroom_median':0.276421,'pos_hotel_isvendor_min':0.2777323333333333,'nag_countryid_hotelbelongto_var':0.27536099999999997,'nag_countryid_isvendor_min':0.2750923333333333,'pos_masterbasicroomid_isebookinghtl_mean':0.27599966666666664,'nag_countryid_ordadvanceday_min':0.27498,'pos_city_isvendor_median':0.27472799999999997,'nag_masterhotelid_isvendor_min':0.2756633333333333,'nag_countryid_hotelbelongto_median':0.2750093333333334,'nag_city_isebookinghtl_mean':0.2759566666666667,'pos_masterhotelid_orderdate_mon_max':0.27624,'pos_masterhotelid_hotelbelongto_var':0.2773393333333333,'pos_masterbasicroomid_isebookinghtl_median':0.2759496666666667,'pos_countryid_arrival_mon_median_sub':0.27516566666666664,'nag_city_hotelbelongto_median':0.27562533333333333,'pos_masterhotelid_isholdroom_median':0.2757366666666667,'pos_hotel_supplierchannel_mean_sub':0.2784356666666667,'nag_hotel_isebookinghtl_var':0.276742,'pos_masterbasicroomid_isholdroom_median_sub':0.2760843333333334,'pos_masterhotelid_orderdate_mon_min':0.27572433333333335,'pos_countryid_isholdroom_var':0.2758633333333333,'nag_hotel_supplierchannel_mean':0.277642,'pos_hotel_ordadvanceday_var':0.277811,'nag_city_arrival_mon_max':0.276345,'nag_supplierid_arrival_mon_mean':0.275664,'nag_room_arrival_mon_mean_sub':0.274626,'pos_masterhotelid_hotelbelongto_mean_sub':0.2764626666666667,'nag_countryid_hotelstar_var':0.27473400000000003,'nag_room_orderdate_mon_var':0.27549966666666664,'nag_room_isebookinghtl_min':0.275455,'nag_masterhotelid_isebookinghtl_max':0.27612600000000004,'pos_countryid_ordadvanceday_median_sub':0.2763116666666667,'pos_hotel_ordadvanceday_mean_sub':0.27684966666666666,'nag_supplierid_isvendor_max':0.27529966666666666,'pos_hotel_etd_mon_mean':0.27743266666666666,'nag_hotel_isvendor_min':0.2759426666666667,'nag_masterhotelid_etd_mon_min':0.2765136666666667,'pos_hotel_hotelbelongto_median_sub':0.2784356666666667,'pos_masterbasicroomid_isholdroom_min':0.274692,'pos_city_arrival_mon_var':0.2754003333333333,'pos_supplierid_hotelstar_median':0.27586133333333335,'pos_supplierid_isvendor_median_sub':0.27500966666666665,'pos_room_supplierchannel_max':0.27659100000000003,'nag_room_arrival_mon_median':0.275587,'nag_masterhotelid_orderdate_mon_min':0.27640800000000004,'nag_city_etd_mon_min':0.275987,'nag_supplierid_etd_mon_mean':0.27674366666666667,'pos_city_orderdate_mon_mean_sub':0.27495933333333333,'pos_supplierid_ordadvanceday_median_sub':0.27521066666666666,'pos_countryid_ordadvanceday_max':0.2748126666666667,'pos_countryid_etd_mon_max':0.27485199999999993,'pos_countryid_isebookinghtl_max':0.275252,'pos_countryid_arrival_mon_mean':0.27499866666666667,'nag_hotel_supplierchannel_median':0.277642,'pos_hotel_arrival_mon_mean':0.278612,'pos_masterhotelid_etd_mon_max':0.27718,'nag_masterbasicroomid_isholdroom_median_sub':0.275727,'pos_room_isvendor_max':0.27513666666666664,'pos_countryid_isvendor_min':0.27610633333333334,'pos_masterbasicroomid_isvendor_max':0.2753466666666667,'pos_masterbasicroomid_hotelbelongto_max':0.274993,'pos_countryid_ordadvanceday_mean':0.27494900000000005,'pos_masterhotelid_etd_mon_median':0.2751963333333333,'pos_masterbasicroomid_supplierchannel_mean':0.275616,'pos_room_isebookinghtl_median_sub':0.27637066666666665,'nag_masterbasicroomid_hotelbelongto_min':0.275619,'pos_countryid_isvendor_mean_sub':0.27649300000000004,'pos_masterhotelid_hotelbelongto_max':0.2763756666666667,'nag_masterbasicroomid_ordadvanceday_max':0.27552133333333334,'nag_supplierid_supplierchannel_mean_sub':0.27595499999999995,'nag_room_ordadvanceday_var':0.2740316666666667,'nag_room_hotelbelongto_median_sub':0.276321,'pos_masterbasicroomid_hotelbelongto_median_sub':0.27453866666666665,'nag_countryid_ordadvanceday_median_sub':0.2754923333333333,'pos_masterhotelid_ordadvanceday_max':0.276829,'nag_masterhotelid_isebookinghtl_median':0.2769066666666667,'nag_room_orderdate_mon_median':0.2741193333333333,'nag_room_supplierchannel_median_sub':0.276321,'nag_hotel_isvendor_mean':0.2767996666666667,'pos_countryid_ordadvanceday_var':0.27545800000000004,'nag_countryid_orderdate_mon_max':0.27506800000000003,'nag_supplierid_isholdroom_median_sub':0.2746866666666667,'pos_hotel_arrival_mon_min':0.277422,'pos_city_isvendor_median_sub':0.27554966666666664,'nag_masterhotelid_hotelstar_median':0.27611499999999994,'nag_masterhotelid_hotelbelongto_mean':0.2767096666666667,'pos_masterhotelid_ordadvanceday_median':0.27638466666666667,'pos_room_arrival_mon_var':0.274047,'pos_hotel_isholdroom_max':0.2775436666666667,'etd_mon_nag_hot':0.2751073333333333,'pos_city_isholdroom_mean_sub':0.27524933333333335,'nag_hotel_etd_mon_mean':0.27598466666666666,'nag_supplierid_isholdroom_median':0.2749796666666667,'nag_masterhotelid_isvendor_max':0.2756593333333333,'nag_room_isvendor_mean':0.276051,'nag_masterbasicroomid_isvendor_mean':0.2753583333333333,'nag_supplierid_arrival_mon_max':0.275873,'pos_masterhotelid_isholdroom_max':0.27571933333333337,'nag_supplierid_isholdroom_mean':0.27552333333333334,'pos_supplierid_isebookinghtl_min':0.27552533333333334,'nag_room_isvendor_median':0.276051,'pos_room_hotelbelongto_max':0.2761636666666667,'nag_city_hotelstar_median':0.27558333333333335,'pos_supplierid_ordadvanceday_mean':0.2774813333333333,'pos_room_hotelstar_median_sub':0.27637066666666665,'nag_room_hotelstar_median_sub':0.276321,'pos_countryid_hotelstar_median_sub':0.276531,'nag_city_orderdate_mon_median':0.2762633333333333,'nag_supplierid_ordadvanceday_max':0.275663,'pos_countryid_orderdate_mon_max':0.275252,'pos_hotel_etd_mon_max':0.278231,'pos_hotel_supplierchannel_median':0.2773113333333333,'nag_city_hotelbelongto_min':0.27570500000000003,'nag_masterbasicroomid_supplierchannel_median_sub':0.275721,'pos_masterbasicroomid_supplierchannel_var':0.27455466666666667,'nag_countryid_orderdate_mon_median':0.275879,'pos_supplierid_etd_mon_var':0.27621,'nag_countryid_isholdroom_median_sub':0.2749836666666667,'pos_city_etd_mon_var':0.275232,'pos_masterhotelid_arrival_mon_max':0.2762343333333333,'pos_masterbasicroomid_orderdate_mon_min':0.27574533333333334,'pos_room_supplierchannel_min':0.27659100000000003,'nag_hotel_isholdroom_mean_sub':0.2767536666666667,'pos_masterbasicroomid_orderdate_mon_max':0.27504666666666666,'nag_city_ordadvanceday_max':0.27608333333333335,'nag_hotel_supplierchannel_var':0.276742,'pos_masterbasicroomid_etd_mon_mean':0.27490466666666663,'nag_masterhotelid_etd_mon_mean_sub':0.27585433333333337,'nag_masterhotelid_isebookinghtl_var':0.2757473333333333,'nag_masterbasicroomid_supplierchannel_median':0.276258,'pos_hotel_arrival_mon_median_sub':0.27738066666666666,'pos_room_hotelbelongto_min':0.2761636666666667,'nag_masterbasicroomid_isholdroom_min':0.27509433333333333,'pos_hotel_ordadvanceday_min':0.2772846666666667,'pos_hotel_isebookinghtl_mean':0.2769176666666667,'pos_hotel_isebookinghtl_max':0.2769176666666667,'pos_room_ordadvanceday_median':0.27520133333333335,'pos_city_hotelbelongto_median':0.27664300000000003,'pos_hotel_hotelbelongto_median':0.27786,'nag_hotel_supplierchannel_median_sub':0.27764633333333333,'nag_masterbasicroomid_isebookinghtl_max':0.2760613333333333,'nag_supplierid_hotelstar_mean_sub':0.27513766666666667,'nag_masterhotelid_ordadvanceday_median_sub':0.27648633333333333,'supplierchannel_pos_hot':0.2764236666666667,'nag_masterbasicroomid_hotelbelongto_mean':0.27660599999999996,'nag_room_isvendor_mean_sub':0.276321,'pos_countryid_supplierchannel_mean_sub':0.2752946666666667,'pos_masterbasicroomid_ordadvanceday_min':0.27566833333333335,'nag_masterhotelid_arrival_mon_max':0.278065,'nag_masterbasicroomid_isebookinghtl_var':0.2759506666666667,'pos_masterbasicroomid_isvendor_median_sub':0.275031,'nag_supplierid_isholdroom_min':0.2750126666666666,'nag_masterbasicroomid_hotelstar_median_sub':0.2760126666666667,'pos_masterbasicroomid_ordadvanceday_max':0.275564,'pos_city_isvendor_max':0.2754773333333333,'pos_countryid_isvendor_median':0.2758,'pos_masterhotelid_hotelbelongto_median':0.2763073333333333,'nag_countryid_hotelstar_mean_sub':0.27478733333333333,'nag_supplierid_hotelstar_min':0.27532566666666664,'pos_masterbasicroomid_ordadvanceday_var':0.2750863333333333,'pos_city_supplierchannel_max':0.27488633333333334,'pos_supplierid_orderdate_mon_mean':0.27691133333333334,'nag_masterhotelid_etd_mon_mean':0.27610666666666667,'pos_countryid_hotelbelongto_min':0.275358,'pos_masterhotelid_hotelbelongto_min':0.276067,'pos_masterbasicroomid_supplierchannel_mean_sub':0.27575900000000003,'nag_room_etd_mon_mean':0.27438066666666666,'pos_room_etd_mon_var':0.27422066666666667,'pos_city_orderdate_mon_mean':0.275357,'pos_hotel_isholdroom_mean':0.2784816666666667,'pos_room_etd_mon_median_sub':0.27618066666666663,'pos_hotel_hotelstar_mean_sub':0.2784356666666667,'pos_countryid_ordadvanceday_median':0.275201,'nag_hotel_ordadvanceday_mean_sub':0.2760953333333333,'nag_masterhotelid_hotelstar_mean_sub':0.276534,'pos_masterbasicroomid_arrival_mon_max':0.27503799999999995,'nag_supplierid_etd_mon_mean_sub':0.27433100000000005,'nag_masterhotelid_isvendor_mean_sub':0.2776146666666666,'nag_masterhotelid_arrival_mon_mean':0.2759546666666667,'pos_city_etd_mon_min':0.2760323333333333,'nag_supplierid_hotelstar_mean':0.27596400000000004,'nag_masterbasicroomid_isholdroom_median':0.27554366666666663,'nag_city_supplierchannel_max':0.27559700000000004,'nag_masterbasicroomid_supplierchannel_mean_sub':0.27470500000000003,'nag_hotel_hotelbelongto_median_sub':0.27764633333333333,'nag_masterbasicroomid_isebookinghtl_mean':0.275944,'nag_room_supplierchannel_max':0.2752113333333333,'pos_countryid_arrival_mon_min':0.27552233333333337,'pos_city_etd_mon_mean':0.276073,'nag_masterbasicroomid_arrival_mon_min':0.276027,'nag_masterbasicroomid_supplierchannel_min':0.2753446666666666,'nag_countryid_isebookinghtl_var':0.2756706666666667,'pos_room_supplierchannel_median':0.27659100000000003,'nag_city_etd_mon_median_sub':0.27482466666666666,'nag_masterhotelid_ordadvanceday_mean_sub':0.2763233333333333,'pos_masterhotelid_orderdate_mon_median_sub':0.275879,'nag_city_isvendor_var':0.27541433333333337,'nag_masterhotelid_isebookinghtl_mean_sub':0.276371,'pos_masterhotelid_hotelstar_median':0.27597166666666667,'nag_hotel_supplierchannel_mean_sub':0.27764633333333333,'nag_masterhotelid_ordadvanceday_max':0.2771726666666667,'pos_masterhotelid_supplierchannel_mean':0.277082,'nag_hotel_supplierchannel_max':0.277642,'nag_room_isholdroom_min':0.2748063333333333,'nag_city_isvendor_median':0.27588166666666664,'pos_hotel_hotelstar_var':0.27649266666666666,'pos_masterbasicroomid_hotelstar_median_sub':0.275195,'nag_countryid_etd_mon_min':0.27565,'nag_room_isholdroom_var':0.274542,'nag_countryid_supplierchannel_median':0.27460100000000004,'pos_hotel_isvendor_max':0.27696066666666663,'nag_masterhotelid_orderdate_mon_median_sub':0.27769133333333335,'pos_masterhotelid_isvendor_mean_sub':0.276761,'nag_masterhotelid_hotelstar_max':0.27658733333333335,'nag_masterbasicroomid_arrival_mon_var':0.27648466666666666,'nag_masterhotelid_hotelbelongto_mean_sub':0.2766573333333333,'nag_supplierid_etd_mon_median':0.27597166666666667,'nag_hotel_arrival_mon_min':0.27701866666666664,'nag_masterbasicroomid_isvendor_min':0.27539933333333333,'nag_hotel_isholdroom_min':0.27781866666666666,'pos_hotel_isebookinghtl_median':0.2769176666666667,'pos_masterbasicroomid_isholdroom_mean_sub':0.275568,'pos_city_hotelbelongto_mean':0.2766233333333334,'pos_masterbasicroomid_isebookinghtl_min':0.2758096666666667,'nag_hotel_hotelstar_var':0.276742,'nag_masterbasicroomid_supplierchannel_mean':0.2757116666666667,'nag_city_ordadvanceday_var':0.27564833333333333,'pos_masterbasicroomid_supplierchannel_median_sub':0.275263,'pos_city_hotelstar_min':0.2755493333333333,'nag_masterbasicroomid_isebookinghtl_min':0.275013,'nag_masterhotelid_supplierchannel_median':0.2755896666666667,'nag_room_hotelstar_var':0.2749486666666667,'nag_countryid_isholdroom_mean':0.27621233333333334,'nag_room_isebookinghtl_mean_sub':0.276321,'nag_masterhotelid_isholdroom_mean_sub':0.2762966666666667,'pos_city_isholdroom_var':0.2755856666666667,'pos_supplierid_supplierchannel_max':0.27518333333333334,'pos_masterbasicroomid_orderdate_mon_median':0.27588133333333326,'nag_masterhotelid_arrival_mon_var':0.277066,'nag_hotel_etd_mon_var':0.27839766666666665,'pos_masterhotelid_hotelstar_var':0.2753276666666667,'nag_countryid_supplierchannel_min':0.27541533333333335,'nag_countryid_isholdroom_mean_sub':0.275244,'nag_countryid_hotelbelongto_max':0.27587433333333333,'nag_masterhotelid_orderdate_mon_mean_sub':0.2767153333333333,'pos_masterbasicroomid_arrival_mon_mean':0.275608,'nag_masterhotelid_arrival_mon_median':0.2768756666666667,'nag_supplierid_isvendor_var':0.2749526666666667,'pos_countryid_isholdroom_mean':0.27504233333333333,'pos_hotel_isholdroom_median':0.27794399999999997,'pos_city_orderdate_mon_var':0.275467,'pos_masterbasicroomid_isholdroom_median':0.2764883333333333,'nag_countryid_supplierchannel_max':0.2742506666666667,'nag_masterhotelid_isholdroom_min':0.27691166666666667,'nag_hotel_isholdroom_median':0.27587433333333333,'pos_masterhotelid_hotelbelongto_median_sub':0.2761073333333333,'nag_masterbasicroomid_hotelbelongto_median':0.27588666666666667,'pos_countryid_isebookinghtl_median_sub':0.27640933333333334,'pos_countryid_supplierchannel_max':0.27582,'nag_countryid_orderdate_mon_mean':0.27558066666666664,'pos_room_etd_mon_mean_sub':0.27515266666666666,'nag_room_isholdroom_mean_sub':0.27572166666666664,'hotelstar_pos_hot':0.2758116666666667,'pos_masterhotelid_isvendor_min':0.276264,'nag_supplierid_ordadvanceday_mean_sub':0.2742116666666667,'nag_city_isholdroom_mean':0.2752446666666667,'pos_countryid_isvendor_median_sub':0.2757826666666667,'nag_countryid_supplierchannel_mean':0.27554833333333334,'nag_room_orderdate_mon_max':0.27593766666666664,'pos_countryid_etd_mon_min':0.27536099999999997,'nag_masterbasicroomid_isebookinghtl_mean_sub':0.2753756666666667,'nag_masterhotelid_supplierchannel_mean':0.2774856666666667,'pos_supplierid_arrival_mon_mean_sub':0.27498833333333333,'nag_supplierid_ordadvanceday_median':0.2774703333333333,'nag_city_supplierchannel_median_sub':0.2743606666666667,'masterbasicroomid_hot':0.2757366666666667,'isholdroom_nag_hot':0.27519166666666667,'nag_room_etd_mon_median':0.2763076666666667,'pos_masterhotelid_isholdroom_min':0.2758026666666667,'pos_city_ordadvanceday_min':0.2746136666666667,'pos_supplierid_isvendor_var':0.275367,'nag_hotel_orderdate_mon_max':0.27716633333333335,'nag_hotel_etd_mon_median':0.277498,'nag_masterbasicroomid_hotelstar_mean_sub':0.275576,'pos_room_isholdroom_var':0.2752793333333333,'nag_countryid_isebookinghtl_min':0.275703,'nag_masterhotelid_ordadvanceday_mean':0.276067,'pos_masterhotelid_orderdate_mon_median':0.2760633333333333,'pos_masterbasicroomid_etd_mon_var':0.27630566666666667,'nag_city_isholdroom_min':0.275858,'pos_city_supplierchannel_var':0.27563866666666664,'pos_supplierid_isebookinghtl_median_sub':0.27534033333333335,'pos_masterbasicroomid_isvendor_mean':0.2750903333333334,'pos_room_etd_mon_min':0.2760253333333333,'nag_masterhotelid_isholdroom_max':0.2747806666666666,'pos_masterbasicroomid_etd_mon_median':0.2760403333333334,'pos_hotel_orderdate_mon_var':0.27797733333333335,'nag_supplierid_hotelbelongto_mean_sub':0.27591233333333337,'nag_city_orderdate_mon_max':0.2748303333333333,'pos_supplierid_hotelbelongto_median':0.2754166666666667,'pos_hotel_ordadvanceday_median_sub':0.27678,'pos_room_arrival_mon_mean':0.27407133333333333,'nag_city_orderdate_mon_mean_sub':0.27549999999999997,'pos_supplierid_hotelstar_max':0.27554300000000004,'nag_room_hotelstar_median':0.2756113333333333,'pos_city_hotelstar_mean':0.276429,'pos_hotel_hotelstar_max':0.27704466666666666,'pos_countryid_orderdate_mon_median_sub':0.27592466666666665,'nag_room_etd_mon_median_sub':0.27493100000000004,'nag_supplierid_hotelbelongto_var':0.27515533333333336,'pos_supplierid_orderdate_mon_var':0.2758123333333333,'nag_city_hotelbelongto_mean':0.2750573333333333,'pos_hotel_isholdroom_median_sub':0.2778133333333333,'nag_city_isebookinghtl_median_sub':0.2751223333333333,'nag_room_supplierchannel_mean_sub':0.276321,'pos_countryid_isvendor_max':0.275252,'pos_supplierid_supplierchannel_var':0.27568733333333334,'pos_masterbasicroomid_etd_mon_mean_sub':0.2749163333333333,'nag_hotel_isvendor_max':0.2771856666666667,'pos_room_etd_mon_median':0.2761263333333333,'pos_city_isebookinghtl_var':0.27533033333333334,'nag_masterhotelid_hotelbelongto_min':0.2763136666666667,'nag_room_hotelstar_mean':0.2756113333333333,'nag_countryid_arrival_mon_median_sub':0.27546933333333334,'nag_city_arrival_mon_var':0.276317,'nag_hotel_isebookinghtl_max':0.2766946666666666,'pos_masterhotelid_hotelstar_min':0.274721,'pos_city_isholdroom_mean':0.2754743333333333,'nag_room_hotelbelongto_median':0.275548,'pos_countryid_hotelbelongto_median':0.27591666666666664,'nag_room_ordadvanceday_median_sub':0.27497600000000005,'nag_hotel_ordadvanceday_max':0.2778566666666667,'pos_masterbasicroomid_isvendor_var':0.27589033333333335,'pos_masterbasicroomid_arrival_mon_min':0.27546966666666667,'pos_hotel_hotelbelongto_mean_sub':0.2784356666666667,'nag_room_arrival_mon_median_sub':0.275071,'pos_supplierid_orderdate_mon_median_sub':0.276422,'nag_masterhotelid_isebookinghtl_min':0.27573966666666666,'pos_masterhotelid_orderdate_mon_mean':0.2766166666666667,'nag_supplierid_orderdate_mon_min':0.275933,'nag_supplierid_arrival_mon_min':0.27577633333333335,'nag_countryid_supplierchannel_mean_sub':0.2762616666666667,'nag_countryid_arrival_mon_min':0.2746283333333333,'pos_hotel_isebookinghtl_median_sub':0.2784356666666667,'pos_city_ordadvanceday_max':0.27553500000000003,'pos_hotel_ordadvanceday_max':0.2784446666666667,'nag_supplierid_hotelstar_median':0.27643733333333337,'nag_city_isvendor_max':0.275268,'pos_city_supplierchannel_median_sub':0.27595733333333333,'pos_masterhotelid_supplierchannel_median':0.2763773333333333,'pos_masterhotelid_isvendor_median_sub':0.2765376666666666,'nag_masterhotelid_arrival_mon_min':0.276707,'nag_countryid_etd_mon_max':0.27529600000000004,'pos_masterbasicroomid_isholdroom_var':0.275275,'nag_masterbasicroomid_isvendor_max':0.27487100000000003,'nag_masterbasicroomid_ordadvanceday_median_sub':0.27431466666666665,'pos_city_isebookinghtl_max':0.2751673333333333,'pos_city_supplierchannel_median':0.275446,'countryid_hot':0.2753033333333333,'nag_supplierid_supplierchannel_median_sub':0.27540733333333334,'nag_masterbasicroomid_orderdate_mon_var':0.2756636666666667,'pos_room_ordadvanceday_mean_sub':0.273077,'nag_countryid_ordadvanceday_var':0.275275,'nag_countryid_isvendor_mean':0.27551966666666666,'nag_masterhotelid_arrival_mon_mean_sub':0.275036,'hotelbelongto_nag_hot':0.274885,'pos_masterbasicroomid_isebookinghtl_median_sub':0.2760106666666666,'pos_supplierid_isvendor_mean':0.275483,'pos_hotel_ordadvanceday_mean':0.27793266666666666,'pos_masterhotelid_isholdroom_median_sub':0.2762096666666667,'pos_countryid_hotelstar_max':0.27526,'nag_room_hotelstar_max':0.2756113333333333,'nag_hotel_hotelbelongto_var':0.276742,'nag_hotel_isholdroom_median_sub':0.2758906666666667,'pos_city_isvendor_mean_sub':0.2751113333333333,'pos_masterbasicroomid_isvendor_median':0.275625,'nag_hotel_hotelstar_median':0.275517,'nag_supplierid_isebookinghtl_median':0.27542200000000006,'pos_masterbasicroomid_hotelstar_mean':0.275161,'pos_masterbasicroomid_hotelstar_mean_sub':0.27510533333333337,'nag_masterhotelid_etd_mon_var':0.27766833333333335,'pos_countryid_isebookinghtl_mean':0.2752383333333333,'nag_supplierid_orderdate_mon_mean_sub':0.276325,'pos_countryid_orderdate_mon_median':0.275893,'pos_hotel_orderdate_mon_median':0.278086,'nag_city_etd_mon_mean_sub':0.274882,'nag_masterbasicroomid_hotelbelongto_var':0.2757563333333333,'nag_masterbasicroomid_ordadvanceday_min':0.27492733333333336,'nag_hotel_isvendor_median':0.27658866666666665,'nag_masterbasicroomid_isvendor_median':0.27496133333333334,'nag_supplierid_isebookinghtl_median_sub':0.2752773333333333,'pos_supplierid_hotelstar_var':0.27670333333333336,'nag_countryid_isholdroom_min':0.27529166666666666,'nag_masterbasicroomid_isebookinghtl_median':0.274623,'nag_room_arrival_mon_mean':0.275322,'nag_masterbasicroomid_orderdate_mon_median_sub':0.2754666666666667,'pos_masterhotelid_arrival_mon_median_sub':0.27570066666666665,'nag_room_etd_mon_min':0.27506333333333327,'nag_supplierid_isholdroom_var':0.27511066666666667,'nag_supplierid_ordadvanceday_var':0.27629333333333334,'nag_hotel_hotelstar_min':0.275517,'pos_masterbasicroomid_hotelbelongto_var':0.275517,'nag_hotel_etd_mon_min':0.275837,'pos_countryid_supplierchannel_mean':0.27440733333333334,'pos_countryid_isholdroom_max':0.27530233333333337,'pos_masterbasicroomid_ordadvanceday_mean_sub':0.27515666666666666,'nag_masterbasicroomid_ordadvanceday_mean_sub':0.2750173333333333,'nag_supplierid_supplierchannel_median':0.27433833333333335,'nag_countryid_hotelstar_mean':0.27521966666666664,'nag_city_hotelbelongto_var':0.27463766666666667,'pos_supplierid_isholdroom_median_sub':0.2758333333333333,'nag_hotel_ordadvanceday_var':0.2777083333333334,'nag_masterhotelid_isvendor_var':0.2759683333333333,'pos_hotel_orderdate_mon_mean':0.276893,'nag_hotel_ordadvanceday_median_sub':0.27713866666666664,'nag_supplierid_isebookinghtl_max':0.2753386666666667,'pos_hotel_isholdroom_mean_sub':0.2790303333333333,'pos_masterhotelid_supplierchannel_min':0.27535000000000004,'pos_countryid_etd_mon_median_sub':0.27533399999999997,'ordadvanceday_nag_hot':0.274935,'pos_city_orderdate_mon_median':0.2749706666666667,'pos_masterhotelid_etd_mon_var':0.27642133333333335,'pos_city_isebookinghtl_mean':0.2749956666666667,'pos_masterhotelid_arrival_mon_var':0.2754353333333333,'pos_room_arrival_mon_min':0.27509066666666665,'nag_countryid_orderdate_mon_min':0.2754716666666667,'nag_supplierid_hotelstar_max':0.27532366666666663,'pos_masterhotelid_ordadvanceday_median_sub':0.27540366666666666,'pos_countryid_etd_mon_var':0.2759863333333333,'nag_masterhotelid_isvendor_mean':0.276866,'nag_city_isvendor_mean_sub':0.275483,'pos_masterbasicroomid_isebookinghtl_var':0.2763926666666667,'nag_masterbasicroomid_hotelstar_var':0.27516066666666666,'nag_hotel_arrival_mon_max':0.27781000000000006,'nag_hotel_isholdroom_mean':0.2771226666666667,'pos_supplierid_isebookinghtl_mean_sub':0.274789,'nag_hotel_arrival_mon_median':0.27748500000000004,'pos_supplierid_arrival_mon_median_sub':0.275773,'pos_countryid_isholdroom_median':0.27532766666666664,'nag_countryid_ordadvanceday_mean_sub':0.274754,'nag_hotel_orderdate_mon_median':0.277409,'nag_countryid_isebookinghtl_max':0.2754493333333333,'pos_room_hotelstar_mean':0.276022,'pos_city_orderdate_mon_max':0.2754643333333333,'nag_supplierid_etd_mon_var':0.2763413333333333,'nag_room_isvendor_max':0.276051,'nag_masterbasicroomid_etd_mon_median':0.27461433333333335,'pos_countryid_isholdroom_median_sub':0.27513533333333334,'pos_hotel_arrival_mon_median':0.277114,'pos_supplierid_isholdroom_mean':0.276384,'pos_room_isvendor_median':0.27513666666666664,'pos_room_isvendor_mean_sub':0.27637066666666665,'pos_masterhotelid_etd_mon_min':0.27654133333333336,'pos_supplierid_arrival_mon_max':0.27698566666666663,'pos_masterbasicroomid_supplierchannel_median':0.27563566666666667,'nag_hotel_orderdate_mon_min':0.27723166666666665,'nag_supplierid_supplierchannel_min':0.2756406666666667,'nag_room_hotelstar_mean_sub':0.276321,'nag_masterbasicroomid_etd_mon_mean':0.27472399999999997,'pos_city_arrival_mon_max':0.275216,'pos_masterbasicroomid_ordadvanceday_median':0.275644,'pos_masterbasicroomid_orderdate_mon_mean':0.27519199999999994,'pos_supplierid_orderdate_mon_min':0.2750116666666667,'pos_hotel_orderdate_mon_median_sub':0.2780023333333333,'nag_hotel_isvendor_median_sub':0.2767253333333333,'nag_masterhotelid_isholdroom_median_sub':0.2766096666666667,'nag_room_hotelbelongto_max':0.275548,'nag_masterhotelid_supplierchannel_median_sub':0.27608966666666673,'nag_hotel_orderdate_mon_median_sub':0.276021,'hotelbelongto_pos_hot':0.27450100000000005,'nag_supplierid_ordadvanceday_min':0.27562366666666666,'pos_countryid_hotelbelongto_max':0.275252,'pos_city_orderdate_mon_median_sub':0.2741423333333333,'pos_hotel_arrival_mon_var':0.27737399999999995,'pos_supplierid_orderdate_mon_median':0.27599466666666667,'pos_room_isebookinghtl_median':0.27604666666666666,'nag_masterhotelid_isebookinghtl_median_sub':0.27617033333333335,'nag_supplierid_supplierchannel_var':0.2756956666666667,'pos_supplierid_isvendor_max':0.2750263333333333,'pos_supplierid_isholdroom_mean_sub':0.27545533333333333,'nag_hotel_etd_mon_max':0.2775843333333334,'nag_supplierid_orderdate_mon_median':0.27615166666666663,'nag_masterbasicroomid_isebookinghtl_median_sub':0.27517600000000003,'nag_masterhotelid_hotelbelongto_max':0.27636533333333335,'pos_masterbasicroomid_hotelstar_min':0.27535766666666667,'pos_city_hotelstar_mean_sub':0.275823,'pos_supplierid_hotelbelongto_var':0.27631066666666665,'pos_room_isvendor_min':0.27513666666666664,'pos_room_hotelbelongto_median_sub':0.27637066666666665,'pos_masterbasicroomid_ordadvanceday_mean':0.27625933333333336,'nag_masterbasicroomid_isvendor_var':0.27596533333333334,'nag_hotel_hotelbelongto_mean_sub':0.27764633333333333,'nag_countryid_hotelbelongto_median_sub':0.27521100000000004,'nag_supplierid_supplierchannel_mean':0.276097,'pos_city_hotelstar_max':0.275634,'nag_masterbasicroomid_isholdroom_var':0.27525466666666665,'nag_masterbasicroomid_etd_mon_median_sub':0.27426733333333336,'pos_hotel_etd_mon_median':0.277601,'nag_supplierid_isvendor_min':0.27607133333333334,'pos_city_ordadvanceday_var':0.27620833333333333,'nag_supplierid_hotelbelongto_min':0.27504100000000004,'nag_countryid_ordadvanceday_max':0.27550399999999997,'pos_city_arrival_mon_mean':0.27585333333333334,'nag_countryid_hotelbelongto_mean_sub':0.27602333333333334,'pos_masterhotelid_orderdate_mon_mean_sub':0.2767543333333333,'pos_supplierid_hotelbelongto_mean':0.27573866666666663,'nag_countryid_hotelstar_max':0.274909,'pos_room_hotelbelongto_mean_sub':0.27637066666666665,'pos_supplierid_arrival_mon_median':0.27599033333333334,'pos_room_orderdate_mon_mean_sub':0.2751516666666667,'pos_countryid_orderdate_mon_min':0.27518933333333334,'nag_city_ordadvanceday_mean_sub':0.27511599999999997,'pos_room_orderdate_mon_var':0.2736783333333333,'pos_hotel_etd_mon_var':0.2770376666666667,'nag_countryid_isebookinghtl_median_sub':0.2747243333333333,'nag_supplierid_hotelstar_median_sub':0.2758986666666667,'pos_room_orderdate_mon_mean':0.27514466666666665,'pos_masterhotelid_isholdroom_mean':0.2759583333333333,'pos_masterbasicroomid_supplierchannel_min':0.275187,'nag_masterhotelid_etd_mon_median':0.27717933333333333,'nag_masterbasicroomid_hotelstar_mean':0.2756266666666667,'pos_masterhotelid_isebookinghtl_var':0.275399,'pos_hotel_isvendor_median':0.2772136666666667,'hotel_hot':0.2796703333333333,'nag_masterbasicroomid_hotelbelongto_max':0.276181,'nag_hotel_arrival_mon_mean_sub':0.276558,'nag_supplierid_arrival_mon_mean_sub':0.2752773333333333,'nag_hotel_hotelstar_mean':0.275517,'nag_city_supplierchannel_var':0.2750263333333334,'nag_city_isvendor_mean':0.27571133333333336,'pos_supplierid_isebookinghtl_mean':0.2760206666666667,'nag_countryid_orderdate_mon_mean_sub':0.27541533333333335,'pos_countryid_orderdate_mon_mean_sub':0.275905,'isebookinghtl_nag_hot':0.27528566666666665,'nag_hotel_arrival_mon_median_sub':0.277094,'pos_masterhotelid_etd_mon_median_sub':0.27535666666666664,'pos_room_isebookinghtl_max':0.27604666666666666,'pos_city_ordadvanceday_mean_sub':0.27373033333333335,'nag_masterbasicroomid_arrival_mon_median_sub':0.27437433333333333,'pos_masterhotelid_isebookinghtl_mean':0.2767463333333333,'nag_countryid_hotelstar_median':0.27600399999999997,'pos_city_orderdate_mon_min':0.275722,'isvendor_nag_hot':0.2738316666666667,'pos_masterhotelid_isvendor_mean':0.276475,'pos_countryid_isebookinghtl_var':0.27450833333333335,'pos_hotel_isebookinghtl_mean_sub':0.2784356666666667,'pos_masterbasicroomid_orderdate_mon_mean_sub':0.27569333333333335,'pos_masterhotelid_isholdroom_var':0.27562166666666665,'pos_room_isholdroom_median_sub':0.2768523333333333,'pos_supplierid_ordadvanceday_var':0.2762446666666667,'nag_countryid_arrival_mon_median':0.2750783333333333,'pos_masterhotelid_ordadvanceday_mean':0.27652600000000005,'nag_supplierid_isvendor_median_sub':0.27491333333333334,'nag_countryid_hotelstar_min':0.27570266666666665,'nag_room_arrival_mon_var':0.2752503333333333,'nag_room_hotelbelongto_mean':0.275548,'pos_city_hotelbelongto_mean_sub':0.2749103333333333,'nag_room_orderdate_mon_mean_sub':0.27447366666666667,'nag_countryid_isebookinghtl_median':0.2757146666666666,'pos_hotel_supplierchannel_min':0.2773113333333333,'pos_countryid_arrival_mon_mean_sub':0.275414,'pos_room_arrival_mon_max':0.27586499999999997,'nag_supplierid_arrival_mon_var':0.27636066666666664,'nag_city_isebookinghtl_max':0.275438,'pos_masterbasicroomid_isholdroom_mean':0.2743393333333333,'nag_city_isholdroom_mean_sub':0.27566833333333335,'pos_countryid_isholdroom_mean_sub':0.27595000000000003,'pos_hotel_hotelbelongto_var':0.27649266666666666,'pos_supplierid_supplierchannel_median_sub':0.27587533333333336,'nag_room_ordadvanceday_min':0.274913,'pos_city_isebookinghtl_median_sub':0.2759226666666666,'nag_masterhotelid_ordadvanceday_var':0.27694066666666667,'pos_supplierid_supplierchannel_mean_sub':0.275592,'pos_room_arrival_mon_median_sub':0.276298,'pos_hotel_hotelstar_mean':0.27704466666666666,'nag_countryid_orderdate_mon_median_sub':0.27559,'nag_countryid_ordadvanceday_mean':0.275579,'nag_hotel_etd_mon_median_sub':0.27647700000000003,'pos_hotel_isholdroom_min':0.27736066666666664,'nag_masterhotelid_orderdate_mon_median':0.27642300000000003,'pos_supplierid_etd_mon_median_sub':0.2749413333333333,'nag_masterbasicroomid_hotelbelongto_mean_sub':0.275063,'pos_room_isholdroom_mean':0.2766693333333334,'nag_room_isholdroom_median_sub':0.2754313333333333,'nag_masterbasicroomid_isvendor_median_sub':0.27596766666666667,'pos_city_hotelbelongto_median_sub':0.275445,'pos_masterhotelid_hotelbelongto_mean':0.277132,'pos_city_arrival_mon_median':0.27475499999999997,'pos_room_orderdate_mon_median_sub':0.2758153333333333,'nag_city_ordadvanceday_median_sub':0.2754196666666667,'nag_city_hotelstar_max':0.27476000000000006,'pos_city_isebookinghtl_mean_sub':0.275561,'nag_supplierid_isebookinghtl_var':0.27483166666666664,'nag_countryid_arrival_mon_mean_sub':0.27465533333333336,'nag_masterbasicroomid_supplierchannel_var':0.2750833333333333,'nag_hotel_isholdroom_var':0.277635,'nag_hotel_hotelbelongto_median':0.276386,'nag_masterbasicroomid_hotelstar_max':0.27601400000000004,'nag_city_hotelstar_var':0.27561966666666665,'pos_masterhotelid_etd_mon_mean':0.275789,'nag_supplierid_etd_mon_min':0.27575466666666665,'pos_city_arrival_mon_min':0.27587466666666666,'pos_masterhotelid_arrival_mon_mean_sub':0.27497900000000003,'nag_supplierid_isvendor_mean':0.27509500000000003,'pos_supplierid_ordadvanceday_mean_sub':0.274062,'nag_countryid_etd_mon_median_sub':0.27598433333333333,'nag_masterhotelid_isvendor_median_sub':0.275847,'pos_room_etd_mon_mean':0.274903,'nag_countryid_supplierchannel_var':0.275503,'pos_city_isebookinghtl_median':0.27565033333333333,'pos_masterbasicroomid_etd_mon_min':0.275587,'nag_masterbasicroomid_orderdate_mon_mean':0.27482366666666663,'room_hot':0.277136,'pos_supplierid_isholdroom_max':0.2756536666666667,'pos_countryid_hotelstar_mean':0.27632033333333333,'nag_room_etd_mon_mean_sub':0.275339,'nag_supplierid_etd_mon_max':0.27558299999999997,'nag_city_ordadvanceday_median':0.2755,'pos_masterbasicroomid_isholdroom_max':0.2753236666666667,'nag_masterbasicroomid_isholdroom_mean':0.27540666666666663,'pos_hotel_ordadvanceday_median':0.2786533333333333,'nag_city_supplierchannel_mean_sub':0.27509066666666665,'nag_supplierid_isebookinghtl_min':0.275892,'pos_city_isvendor_var':0.274982,'pos_hotel_hotelbelongto_mean':0.27786,'nag_masterbasicroomid_isvendor_mean_sub':0.27516266666666667,'nag_masterbasicroomid_orderdate_mon_mean_sub':0.275232,'pos_supplierid_supplierchannel_mean':0.2753216666666667,'pos_masterbasicroomid_arrival_mon_median':0.27613866666666664,'pos_room_isebookinghtl_mean_sub':0.27637066666666665,'nag_city_isebookinghtl_median':0.2758696666666667,'isvendor_pos_hot':0.2738316666666667,'pos_countryid_supplierchannel_median':0.2757056666666667,'pos_room_isvendor_var':0.27457233333333336,'nag_countryid_etd_mon_mean_sub':0.27504599999999996,'nag_hotel_ordadvanceday_mean':0.2761686666666667,'pos_city_hotelbelongto_min':0.27491166666666667,'pos_room_hotelbelongto_var':0.27457233333333336,'nag_supplierid_supplierchannel_max':0.27433833333333335,'pos_countryid_isvendor_mean':0.2755533333333334,'pos_supplierid_hotelstar_mean_sub':0.27542133333333335,'nag_masterbasicroomid_arrival_mon_median':0.2755183333333333,'nag_countryid_etd_mon_mean':0.27498399999999995,'nag_countryid_etd_mon_var':0.27549833333333335,'pos_countryid_isholdroom_min':0.275252,'nag_supplierid_hotelstar_var':0.276538,'pos_countryid_supplierchannel_var':0.27480866666666665,'city_hot':0.2750886666666667,'nag_masterbasicroomid_ordadvanceday_mean':0.27494899999999994,'pos_masterhotelid_isebookinghtl_min':0.27530933333333335,'nag_masterbasicroomid_orderdate_mon_min':0.27540233333333336,'nag_city_orderdate_mon_mean':0.275522,'pos_hotel_isholdroom_var':0.27766699999999994,'nag_masterhotelid_supplierchannel_var':0.276021,'pos_masterhotelid_arrival_mon_min':0.27663366666666667,'nag_room_ordadvanceday_mean_sub':0.27463533333333334,'nag_city_supplierchannel_min':0.27576033333333333,'nag_countryid_isvendor_mean_sub':0.275526,'nag_supplierid_hotelbelongto_median':0.27559833333333333,'pos_masterhotelid_isebookinghtl_max':0.2758933333333333,'pos_masterbasicroomid_etd_mon_max':0.2752343333333333,'nag_masterhotelid_isebookinghtl_mean':0.2770803333333333,'nag_room_isholdroom_mean':0.275353,'nag_masterhotelid_hotelbelongto_median':0.2758336666666667,'nag_city_orderdate_mon_median_sub':0.2763963333333333,'nag_room_arrival_mon_max':0.275432,'pos_supplierid_hotelbelongto_max':0.27542,'pos_masterbasicroomid_arrival_mon_mean_sub':0.27497366666666667,'pos_city_isvendor_min':0.27467033333333335,'nag_masterhotelid_hotelstar_min':0.27542,'pos_supplierid_arrival_mon_var':0.27620400000000006,'pos_city_isebookinghtl_min':0.27563,'pos_countryid_hotelstar_var':0.2754493333333333,'pos_countryid_arrival_mon_median':0.27552499999999996,'pos_masterbasicroomid_hotelstar_var':0.275914,'nag_masterhotelid_arrival_mon_median_sub':0.27666433333333335,'pos_supplierid_etd_mon_median':0.27607,'nag_supplierid_etd_mon_median_sub':0.27544266666666667,'pos_countryid_hotelbelongto_mean_sub':0.27651733333333334,'isholdroom_pos_hot':0.27519166666666667,'pos_countryid_arrival_mon_max':0.275445,'nag_room_isebookinghtl_var':0.2749486666666667,'pos_room_arrival_mon_mean_sub':0.2739756666666667,'pos_countryid_etd_mon_mean_sub':0.275752,'pos_masterhotelid_hotelstar_median_sub':0.2755546666666666,'nag_masterbasicroomid_ordadvanceday_var':0.27529366666666666,'orderdate_mon_nag_hot':0.2750456666666667,'nag_city_isholdroom_var':0.27654566666666663,'pos_masterhotelid_isebookinghtl_median':0.27609900000000004,'pos_room_hotelstar_min':0.276022,'nag_city_arrival_mon_mean':0.2754203333333333,'pos_hotel_orderdate_mon_max':0.2786996666666666}
len(feature_performance)

1000

In [8]:
feature_performance2={'bk_num': 0.27546133333333334,'zq_num': 0.2760476666666667,'city_his_daysum':0.2753523333333333,'city_num':0.2751226666666667,'cityorder_maxdate_sub':0.27530266666666664,'cityorder_maxdate_sub2':0.2746043333333334,'cityorder_mindate_sub':0.276306,'cityorder_mindate_sub2':0.27344233333333334,'countryid_his_daysum':0.27503733333333336,'countryid_num':0.2756436666666667,'countryidorder_maxdate_sub2':0.27323566666666665,'countryidorder_mindate_sub':0.2758123333333333,'countryidorder_mindate_sub2':0.2730356666666666,'hotel_his_daysum':0.27748866666666666,'hotel_num':0.277881,'hotelorder_maxdate_sub':0.27702699999999997,'hotelorder_maxdate_sub2':0.274121,'hotelorder_mindate_sub':0.2763586666666667,'hotelorder_mindate_sub2':0.27247166666666667,'masterbasicroomid_his_daysum':0.2758783333333333,'masterbasicroomid_num':0.27571833333333334,'masterbasicroomidorder_maxdate_sub':0.275376,'masterbasicroomidorder_maxdate_sub2':0.2732206666666667,'masterbasicroomidorder_mindate_sub':0.2752386666666667,'masterbasicroomidorder_mindate_sub2':0.2730653333333333,'masterhotelid_his_daysum':0.2756636666666667,'masterhotelid_num':0.276298,'masterhotelidorder_maxdate_sub':0.275515,'masterhotelidorder_maxdate_sub2':0.27264666666666665,'masterhotelidorder_mindate_sub':0.2755073333333333,'masterhotelidorder_mindate_sub2':0.2720453333333333,'room_his_daysum':0.2755823333333333,'room_num':0.2761016666666667,'roomorder_maxdate_sub':0.27449566666666664,'roomorder_maxdate_sub2':0.272675,'roomorder_mindate_sub':0.2752136666666667,'roomorder_mindate_sub2':0.27450366666666665,'supplierid_his_daysum':0.2779206666666667,'supplierid_num':0.2777503333333334,'supplieridorder_maxdate_sub':0.2750956666666667,'supplieridorder_maxdate_sub2':0.2732646666666667,'supplieridorder_mindate_sub':0.2766283333333333,'supplieridorder_mindate_sub2':0.273831}
len(feature_performance2)

43

In [9]:
feature_performance = dict(feature_performance, **feature_performance2 )

In [10]:
good_feature = []
baseline = 0.276

for k,v in sorted(feature_performance.items(), key=lambda x:x[1], reverse=True):
    if v > baseline:
        good_feature.append(k)
len(good_feature)

384

In [41]:
good_feature

['hotel_hot',
 'pos_hotel_isholdroom_mean_sub',
 'pos_hotel_orderdate_mon_max',
 'pos_hotel_ordadvanceday_median',
 'pos_hotel_arrival_mon_mean',
 'pos_hotel_isvendor_mean_sub',
 'pos_hotel_isholdroom_mean',
 'pos_hotel_ordadvanceday_max',
 'pos_hotel_supplierchannel_mean_sub',
 'pos_hotel_isebookinghtl_median_sub',
 'pos_hotel_supplierchannel_median_sub',
 'pos_hotel_hotelstar_median_sub',
 'pos_hotel_hotelbelongto_mean_sub',
 'pos_hotel_hotelbelongto_median_sub',
 'pos_hotel_isebookinghtl_mean_sub',
 'pos_hotel_hotelstar_mean_sub',
 'nag_hotel_etd_mon_var',
 'nag_masterhotelid_etd_mon_max',
 'nag_hotel_orderdate_mon_mean',
 'pos_hotel_etd_mon_max',
 'pos_hotel_etd_mon_min',
 'pos_hotel_orderdate_mon_median',
 'nag_masterhotelid_arrival_mon_max',
 'pos_hotel_orderdate_mon_mean_sub',
 'pos_hotel_arrival_mon_max',
 'pos_hotel_etd_mon_mean_sub',
 'pos_hotel_orderdate_mon_median_sub',
 'pos_hotel_orderdate_mon_var',
 'pos_hotel_isholdroom_median',
 'pos_hotel_ordadvanceday_mean',
 'suppli

In [12]:
good_fea = ['pos_hotel_isholdroom_mean_sub','pos_hotel_orderdate_mon_max','pos_hotel_ordadvanceday_median','pos_hotel_arrival_mon_mean','pos_hotel_isvendor_mean_sub','pos_hotel_isholdroom_mean','pos_hotel_ordadvanceday_max','pos_hotel_supplierchannel_mean_sub','pos_hotel_isebookinghtl_median_sub','pos_hotel_supplierchannel_median_sub','pos_hotel_hotelstar_median_sub','pos_hotel_hotelbelongto_mean_sub','pos_hotel_hotelbelongto_median_sub','pos_hotel_isebookinghtl_mean_sub','pos_hotel_hotelstar_mean_sub','nag_hotel_etd_mon_var','nag_masterhotelid_etd_mon_max','nag_hotel_orderdate_mon_mean','pos_hotel_etd_mon_max','pos_hotel_etd_mon_min','pos_hotel_orderdate_mon_median','nag_masterhotelid_arrival_mon_max','pos_hotel_orderdate_mon_mean_sub','pos_hotel_arrival_mon_max','pos_hotel_etd_mon_mean_sub','pos_hotel_orderdate_mon_median_sub','pos_hotel_orderdate_mon_var','pos_hotel_isholdroom_median','pos_hotel_ordadvanceday_mean','supplierid_his_daysum','hotel_num','pos_hotel_hotelbelongto_median','pos_hotel_hotelbelongto_min','pos_hotel_hotelbelongto_mean','nag_hotel_ordadvanceday_max','nag_hotel_isholdroom_min','pos_hotel_isholdroom_median_sub','pos_hotel_ordadvanceday_var','nag_hotel_arrival_mon_max','nag_hotel_isvendor_var','supplierid_num','nag_hotel_orderdate_mon_var','pos_hotel_isvendor_min','nag_hotel_ordadvanceday_var','nag_masterhotelid_orderdate_mon_median_sub','nag_masterhotelid_etd_mon_var','pos_hotel_isholdroom_var','nag_hotel_isebookinghtl_mean_sub','nag_hotel_supplierchannel_median_sub','nag_hotel_hotelstar_median_sub','nag_hotel_hotelbelongto_median_sub','nag_hotel_isebookinghtl_median_sub','nag_hotel_supplierchannel_mean_sub','nag_hotel_hotelbelongto_mean_sub','nag_hotel_hotelstar_mean_sub','nag_hotel_supplierchannel_max','nag_hotel_supplierchannel_mean','nag_hotel_supplierchannel_median','nag_hotel_supplierchannel_min','nag_hotel_isholdroom_var','nag_masterhotelid_isvendor_mean_sub','nag_hotel_orderdate_mon_mean_sub','pos_hotel_etd_mon_median','nag_hotel_etd_mon_max','pos_hotel_isholdroom_max','supplierid_hot','hotel_his_daysum','nag_masterhotelid_supplierchannel_mean','nag_hotel_arrival_mon_median','pos_supplierid_ordadvanceday_mean','nag_supplierid_ordadvanceday_median','pos_hotel_etd_mon_mean','pos_hotel_arrival_mon_min','nag_hotel_orderdate_mon_median','pos_hotel_arrival_mon_median_sub','pos_hotel_arrival_mon_var','pos_hotel_isholdroom_min','nag_hotel_arrival_mon_mean','pos_hotel_etd_mon_median_sub','pos_masterhotelid_hotelbelongto_var','nag_hotel_isholdroom_max','pos_hotel_supplierchannel_median','pos_hotel_supplierchannel_mean','pos_hotel_supplierchannel_min','pos_hotel_supplierchannel_max','pos_hotel_isvendor_mean','pos_hotel_ordadvanceday_min','pos_hotel_orderdate_mon_min','pos_hotel_isvendor_median_sub','nag_hotel_orderdate_mon_min','pos_hotel_isvendor_median','nag_hotel_arrival_mon_var','nag_hotel_isvendor_max','pos_room_isholdroom_max','pos_masterhotelid_etd_mon_max','nag_masterhotelid_etd_mon_median','nag_masterhotelid_ordadvanceday_max','nag_hotel_orderdate_mon_max','pos_masterhotelid_isvendor_var','pos_masterhotelid_supplierchannel_var','nag_masterhotelid_supplierchannel_mean_sub','nag_hotel_ordadvanceday_median_sub','room_hot','pos_masterhotelid_hotelbelongto_mean','nag_hotel_isholdroom_mean','pos_hotel_arrival_mon_median','nag_hotel_arrival_mon_median_sub','pos_masterhotelid_supplierchannel_mean','nag_masterhotelid_isebookinghtl_mean','nag_masterhotelid_arrival_mon_var','pos_hotel_hotelstar_median','pos_hotel_hotelstar_max','pos_hotel_hotelstar_mean','pos_hotel_hotelstar_min','pos_hotel_etd_mon_var','nag_hotel_arrival_mon_min','pos_supplierid_arrival_mon_max','pos_hotel_isvendor_max','nag_masterhotelid_orderdate_mon_mean','nag_masterhotelid_hotelbelongto_var','nag_masterhotelid_ordadvanceday_var','pos_hotel_isvendor_var','pos_hotel_isebookinghtl_median','pos_hotel_isebookinghtl_min','pos_hotel_isebookinghtl_mean','pos_hotel_isebookinghtl_max','nag_masterhotelid_isholdroom_min','pos_supplierid_orderdate_mon_mean','nag_masterhotelid_isebookinghtl_median','pos_hotel_orderdate_mon_mean','nag_masterhotelid_isvendor_mean','pos_room_isholdroom_median_sub','pos_hotel_ordadvanceday_mean_sub','pos_masterhotelid_ordadvanceday_max','nag_hotel_isvendor_mean','pos_hotel_ordadvanceday_median_sub','pos_supplierid_orderdate_mon_mean_sub','pos_masterhotelid_supplierchannel_median_sub','pos_masterhotelid_isvendor_mean_sub','pos_masterhotelid_orderdate_mon_mean_sub','nag_hotel_isholdroom_mean_sub','pos_masterhotelid_isebookinghtl_mean','nag_supplierid_etd_mon_mean','pos_supplierid_arrival_mon_mean','nag_hotel_isebookinghtl_var','nag_hotel_hotelbelongto_var','nag_hotel_hotelstar_var','nag_hotel_supplierchannel_var','nag_hotel_isvendor_median_sub','nag_supplierid_orderdate_mon_var','nag_masterhotelid_orderdate_mon_mean_sub','pos_masterhotelid_isvendor_max','nag_masterhotelid_hotelbelongto_mean','nag_masterhotelid_arrival_mon_min','pos_supplierid_hotelstar_var','nag_hotel_isebookinghtl_median','nag_hotel_isebookinghtl_min','nag_hotel_isebookinghtl_max','nag_hotel_isebookinghtl_mean','pos_masterhotelid_isebookinghtl_mean_sub','nag_hotel_ordadvanceday_median','masterhotelid_hot','pos_room_isholdroom_mean','nag_masterhotelid_arrival_mon_median_sub','nag_masterhotelid_orderdate_mon_var','nag_masterhotelid_hotelbelongto_mean_sub','pos_city_hotelbelongto_median','pos_masterhotelid_arrival_mon_min','supplieridorder_mindate_sub','pos_city_hotelbelongto_mean','pos_masterhotelid_orderdate_mon_mean','nag_masterhotelid_isholdroom_median_sub','nag_masterbasicroomid_hotelbelongto_mean','nag_supplierid_orderdate_mon_median_sub','pos_room_supplierchannel_mean','pos_room_supplierchannel_median','pos_room_supplierchannel_max','pos_room_supplierchannel_min','nag_hotel_isvendor_median','nag_masterhotelid_hotelstar_max','nag_hotel_arrival_mon_mean_sub','nag_city_isholdroom_var','pos_masterhotelid_etd_mon_min','nag_supplierid_hotelstar_var','pos_masterhotelid_isvendor_median_sub','nag_masterhotelid_hotelstar_mean_sub','pos_countryid_hotelstar_median_sub','nag_supplierid_isvendor_median','pos_masterhotelid_ordadvanceday_mean','pos_supplierid_hotelbelongto_min','nag_supplierid_isholdroom_mean_sub','pos_countryid_hotelbelongto_mean_sub','nag_masterhotelid_etd_mon_min','nag_city_arrival_mon_median_sub','pos_countryid_isvendor_mean_sub','pos_hotel_isebookinghtl_var','pos_hotel_supplierchannel_var','pos_hotel_hotelstar_var','pos_hotel_hotelbelongto_var','pos_masterbasicroomid_isholdroom_median','nag_masterhotelid_ordadvanceday_median_sub','pos_supplierid_ordadvanceday_median','nag_masterbasicroomid_arrival_mon_var','pos_countryid_hotelstar_mean_sub','nag_hotel_etd_mon_median_sub','pos_masterhotelid_isvendor_mean','pos_supplierid_etd_mon_mean','pos_masterhotelid_hotelbelongto_mean_sub','nag_masterhotelid_supplierchannel_min','pos_room_orderdate_mon_max','pos_supplierid_hotelstar_mean','nag_supplierid_hotelstar_median','pos_city_hotelstar_mean','pos_room_isholdroom_median','supplierchannel_nag_hot','supplierchannel_pos_hot','nag_masterhotelid_orderdate_mon_median','pos_supplierid_orderdate_mon_median_sub','pos_masterhotelid_etd_mon_var','nag_masterhotelid_isholdroom_median','pos_countryid_isebookinghtl_median_sub','nag_masterhotelid_orderdate_mon_min','nag_city_orderdate_mon_median_sub','pos_masterbasicroomid_isebookinghtl_var','nag_masterbasicroomid_hotelbelongto_median_sub','nag_hotel_hotelbelongto_median','nag_hotel_hotelbelongto_max','nag_hotel_hotelbelongto_min','nag_hotel_hotelbelongto_mean','pos_masterhotelid_ordadvanceday_median','pos_supplierid_isholdroom_mean','pos_masterhotelid_supplierchannel_median','pos_masterhotelid_hotelbelongto_max','nag_masterhotelid_isebookinghtl_mean_sub','pos_room_isebookinghtl_median_sub','pos_room_supplierchannel_mean_sub','pos_room_hotelstar_median_sub','pos_room_isvendor_mean_sub','pos_room_hotelbelongto_mean_sub','pos_room_hotelbelongto_median_sub','pos_room_hotelstar_mean_sub','pos_room_isvendor_median_sub','pos_room_isebookinghtl_mean_sub','pos_room_supplierchannel_median_sub','nag_masterhotelid_hotelbelongto_max','nag_supplierid_isebookinghtl_mean','nag_supplierid_arrival_mon_var','hotelorder_mindate_sub','nag_supplierid_arrival_mon_median','pos_masterhotelid_isebookinghtl_median_sub','nag_city_arrival_mon_max','nag_supplierid_etd_mon_var','pos_room_etd_mon_max','nag_supplierid_orderdate_mon_mean_sub','nag_masterhotelid_ordadvanceday_mean_sub','nag_room_isebookinghtl_median_sub','nag_room_supplierchannel_median_sub','nag_room_hotelstar_mean_sub','nag_room_hotelbelongto_mean_sub','nag_room_hotelstar_median_sub','nag_room_supplierchannel_mean_sub','nag_room_isvendor_median_sub','nag_room_isvendor_mean_sub','nag_room_isebookinghtl_mean_sub','nag_room_hotelbelongto_median_sub','pos_countryid_hotelstar_mean','nag_city_arrival_mon_var','nag_masterhotelid_hotelbelongto_min','pos_countryid_ordadvanceday_median_sub','pos_supplierid_hotelbelongto_var','nag_room_etd_mon_median','pos_masterhotelid_hotelbelongto_median','cityorder_mindate_sub','nag_masterhotelid_isholdroom_mean','pos_masterbasicroomid_etd_mon_var','pos_room_arrival_mon_median_sub','masterhotelid_num','nag_masterhotelid_isholdroom_mean_sub','nag_supplierid_ordadvanceday_var','nag_masterhotelid_hotelbelongto_median_sub','nag_countryid_isholdroom_var','nag_supplierid_ordadvanceday_mean','pos_masterhotelid_isvendor_min','nag_city_orderdate_mon_median','nag_countryid_supplierchannel_mean_sub','pos_masterbasicroomid_ordadvanceday_mean','nag_masterbasicroomid_supplierchannel_median','pos_masterhotelid_supplierchannel_mean_sub','pos_supplierid_ordadvanceday_var','nag_masterhotelid_hotelstar_median_sub','pos_masterhotelid_orderdate_mon_max','pos_supplierid_ordadvanceday_max','pos_masterhotelid_arrival_mon_max','nag_countryid_isebookinghtl_mean_sub','nag_countryid_isholdroom_mean','pos_supplierid_etd_mon_var','pos_masterhotelid_isholdroom_median_sub','pos_city_ordadvanceday_var','pos_masterbasicroomid_isebookinghtl_mean_sub','pos_supplierid_arrival_mon_var','pos_supplierid_isholdroom_var','pos_countryid_hotelbelongto_median_sub','nag_masterbasicroomid_hotelbelongto_max','pos_room_etd_mon_median_sub','nag_masterhotelid_isebookinghtl_median_sub','nag_city_isvendor_median_sub','nag_hotel_ordadvanceday_mean','pos_room_hotelbelongto_median','pos_room_hotelbelongto_min','pos_room_hotelbelongto_max','pos_room_hotelbelongto_mean','pos_supplierid_etd_mon_min','pos_countryid_isebookinghtl_median','nag_supplierid_orderdate_mon_median','pos_masterbasicroomid_arrival_mon_median','pos_city_arrival_mon_mean_sub','pos_countryid_supplierchannel_min','nag_masterhotelid_supplierchannel_max','pos_room_etd_mon_median','nag_masterhotelid_isebookinghtl_max','nag_masterhotelid_hotelstar_median','pos_masterhotelid_hotelbelongto_median_sub','nag_masterhotelid_etd_mon_mean','pos_countryid_isvendor_min','room_num','pos_masterhotelid_isebookinghtl_median','pos_masterhotelid_hotelstar_max','nag_supplierid_supplierchannel_mean','nag_hotel_ordadvanceday_mean_sub','nag_masterhotelid_supplierchannel_median_sub','pos_city_ordadvanceday_median_sub','pos_masterbasicroomid_hotelbelongto_mean_sub','pos_masterbasicroomid_isholdroom_median_sub','nag_city_ordadvanceday_max','nag_masterhotelid_isholdroom_var','pos_city_etd_mon_mean','pos_room_arrival_mon_median','nag_supplierid_isvendor_min','pos_supplierid_etd_mon_median','nag_masterhotelid_ordadvanceday_mean','pos_masterhotelid_hotelbelongto_min','pos_masterhotelid_orderdate_mon_var','pos_masterhotelid_orderdate_mon_median','nag_masterbasicroomid_isebookinghtl_max','nag_city_hotelstar_mean','hotelstar_nag_hot','nag_room_isvendor_median','nag_room_isvendor_min','nag_room_isvendor_mean','nag_room_isvendor_max','zq_num','pos_room_isebookinghtl_min','pos_room_isebookinghtl_max','pos_room_isebookinghtl_mean','pos_room_isebookinghtl_median','pos_supplierid_isvendor_min','pos_masterbasicroomid_etd_mon_median','pos_city_etd_mon_min','pos_masterbasicroomid_ordadvanceday_median_sub','nag_masterbasicroomid_arrival_mon_min','nag_city_isholdroom_max','pos_room_etd_mon_min','nag_countryid_hotelbelongto_mean_sub','pos_supplierid_orderdate_mon_max','pos_room_hotelstar_mean','pos_room_hotelstar_median','pos_room_hotelstar_max','pos_room_hotelstar_min','nag_hotel_orderdate_mon_median_sub','nag_masterhotelid_supplierchannel_var','pos_supplierid_isebookinghtl_mean','nag_masterbasicroomid_hotelstar_max','nag_masterbasicroomid_hotelstar_median_sub','nag_masterhotelid_orderdate_mon_max','pos_masterbasicroomid_isebookinghtl_median_sub','pos_countryid_hotelbelongto_var','nag_countryid_hotelstar_median','pos_supplierid_isholdroom_median','nag_supplierid_hotelbelongto_mean']
len(good_fea)

379

In [ ]:
tmp_predictors = basic_fea.copy()
tmp_round = {}
last_prc = 0.27682866666666667
params = {
    'objective': 'binary:logistic',
    'eta': 0.1,
    'colsample_bytree': 0.886,
    'min_child_weight': 0.1,
    'max_depth': 15,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'seed': 2018,
    'missing': -1
}

for i in tqdm_notebook(range(38)):
    tmp_predictors += good_fea[i*10:(i+1)*10]
    assert(len(tmp_predictors) == 28+(i+1)*10)
    xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])
    cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=20000,
    nfold=3,
    feval=evalerror,
    verbose_eval=0,
    early_stopping_rounds=100,
    seed=2018)
    tmp_round[i] = cv.shape[0]
    print(str(i) + ':' + str(last_prc) + '->' + str(-cv.iloc[-1, :]['test-prc-mean']))
    last_prc = -cv.iloc[-1, :]['test-prc-mean']
    del xgbtrain
    _ = gc.collect()

0:0.27682866666666667->0.28237133333333336
1:0.28237133333333336->0.2848296666666667
2:0.2848296666666667->0.28349033333333334
3:0.28349033333333334->0.284373
4:0.284373->0.28565066666666666
5:0.28565066666666666->0.28414133333333336
6:0.28414133333333336->0.2851106666666667
7:0.2851106666666667->0.28460500000000005
8:0.28460500000000005->0.2864663333333333
9:0.2864663333333333->0.28498566666666664
10:0.28498566666666664->0.28479333333333334
11:0.28479333333333334->0.28398233333333334
12:0.28398233333333334->0.284204
13:0.284204->0.28378533333333333
14:0.28378533333333333->0.284121
15:0.284121->0.283885
16:0.283885->0.2853780000000001
17:0.2853780000000001->0.28556066666666663
18:0.28556066666666663->0.28434833333333337
19:0.28434833333333337->0.28538833333333335
20:0.28538833333333335->0.28494433333333336


In [ ]:
# new
0:0.27682866666666667->0.28237133333333336
# 1:0.28237133333333336->0.2848296666666667
2:0.2848296666666667->0.28349033333333334
3:0.28349033333333334->0.284373
# 4:0.284373->0.28565066666666666
5:0.28565066666666666->0.28414133333333336
6:0.28414133333333336->0.2851106666666667
7:0.2851106666666667->0.28460500000000005
# 8:0.28460500000000005->0.2864663333333333
9:0.2864663333333333->0.28498566666666664
10:0.28498566666666664->0.28479333333333334
11:0.28479333333333334->0.28398233333333334
12:0.28398233333333334->0.284204
13:0.284204->0.28378533333333333
14:0.28378533333333333->0.284121
15:0.284121->0.283885
16:0.283885->0.2853780000000001
# 17:0.2853780000000001->0.28556066666666663
18:0.28556066666666663->0.28434833333333337
19:0.28434833333333337->0.28538833333333335

In [ ]:
# order_id
0:0.27682866666666667->0.283177
# 1:0.283177->0.28551966666666667
2:0.28551966666666667->0.28515
3:0.28515->0.28515433333333334
4:0.28515433333333334->0.2847536666666667
5:0.2847536666666667->0.284173
6:0.284173->0.284559
7:0.284559->0.2844803333333333
8:0.2844803333333333->0.2849216666666667
9:0.2849216666666667->0.2848636666666667
# 10:0.2848636666666667->0.28579166666666667
11:0.28579166666666667->0.28554799999999997
12:0.28554799999999997->0.28517366666666666
13:0.28517366666666666->0.2854986666666666
14:0.2854986666666666->0.285144
15:0.285144->0.28505433333333335
16:0.28505433333333335->0.2837906666666667
17:0.2837906666666667->0.28387766666666664
# 18:0.28387766666666664->0.28577600000000003
19:0.28577600000000003->0.285498
20:0.285498->0.2853463333333333
21:0.2853463333333333->0.28623566666666667
22:0.28623566666666667->0.28608266666666665
23:0.28608266666666665->0.28582066666666667
24:0.28582066666666667->0.28506000000000004
# 25:0.28506000000000004->0.286284
26:0.286284->0.2861983333333334
27:0.2861983333333334->0.2852506666666667
28:0.2852506666666667->0.285058

In [ ]:
0:0.27682866666666667->0.2820586666666667
# 1:0.2820586666666667->0.282586 线上：0.25579215
2:0.282586->0.28242533333333336
3:0.28242533333333336->0.2825243333333333
# 4:0.2825243333333333->0.28330999999999995 线上：0.25725695
5:0.28330999999999995->0.28237
6:0.28237->0.2829953333333333
7:0.2829953333333333->0.282866
8:0.282866->0.28168366666666667
# 9:0.28168366666666667->0.28384666666666664
10:0.28384666666666664->0.28329766666666667
11:0.28329766666666667->0.283422
12:0.283422->0.2826106666666666
13:0.2826106666666666->0.2834453333333333
14:0.2834453333333333->0.283018
15:0.283018->0.2832156666666667
16:0.2832156666666667->0.28277033333333335
17:0.28277033333333335->0.28330266666666665
18:0.28330266666666665->0.2846553333333333
19:0.2846553333333333->0.2841276666666666
# 20:0.2841276666666666->0.2849646666666667 线上：
21:0.2849646666666667->0.28379166666666666
22:0.28379166666666666->0.2837296666666667
23:0.2837296666666667->0.2841606666666667
24:0.2841606666666667->0.28443100000000004
25:0.28443100000000004->0.284716
26:0.284716->0.284556
27:0.284556->0.284855
# 28:0.284855->0.28581233333333333 线上：0.26599329
29:0.28581233333333333->0.283661
30:0.283661->0.2847033333333333
31:0.2847033333333333->0.28402
32:0.28402->0.284575
33:0.284575->0.2841833333333333
34:0.2841833333333333->0.2841516666666667
35:0.2841516666666667->0.2842453333333333
36:0.2842453333333333->0.28344566666666665
37:0.28344566666666665->0.2835703333333333
38:0.2835703333333333->0.28435166666666667